## Modelo Buy Til You Die

Objetivo desta análise é calcular a probabilidade de compra e vida útil do cliente com base em 3 fatores, Recencia, Frequência e Valor monetário de compra, para então sabermos quais os clientes mais valiosos ou não para o negócio e também ter a probabilidade de compra do cliente.

Em seguida vamos unir a pontuação de probabilidade de vida, com outras métricas do cliente como helthscore, taxa de reclamações, problemas com entrega e defeitos de produto e criar um indicador final que reflita toda a saude do cliente e sua probabilidade de churn.

https://pypi.org/project/Lifetimes/0.6.0.0/
https://lifetimes.readthedocs.io/en/latest/Quickstart.html
https://www.youtube.com/watch?v=DJ3r0r1h5Pg

In [1]:
#!pip install redshift_connector
#!pip install lifetimes
#!pip install --upgrade lifetimes

In [2]:
# Importando os pacotes básicos
import pandas as pd
import matplotlib.pyplot as plt
import redshift_connector
import cx_Oracle

## 0 - Sumario

<br/> 1 - Carregando o Dataset. <br/>
<br/> 2 - Tratamento dos dados. <br/>
<br/> 3 - Criando a matrix RFM (Recency, Frequency, Monetary Value). <br/>
<br/> 4 - Fazendo fit do modelo e criando alguns gráficos para análise RFM. <br/>
<br/> 5 - Probabilidade transações futuras com modelo Gamma-Gamma. <br/>
<br/> 6 - Fazendo previsão de transações futuras de um único cliente com .predict. <br/>
<br/> 7 - Filtrando e analisando clientes que fizeram mais de uma compra. <br/>
<br/> 8 - Treinando o modelo Gamma Gamma considerando o valor monetário. <br/>
<br/> 9 - Prevendo a média de valor de transação para cada cliente. <br/>
<br/> 10 - Calculando o CLV dos clientes nos próximos 12 meses. <br/>
<br/> 11 - Gerando um novo dataset com base nos calculos feitos. <br/>

## 1 - Carregando os datasets

In [ ]:
# Connects to Redshift cluster using AWS credentials
conn = redshift_connector.connect(
    host='',
    database='',
    user='',
    password=''
 )

cursor: redshift_connector.Cursor = conn.cursor()


# Dataset 
sql_count_produtos_cli = """
with base_dados as (
SELECT 
f_items.cd_pedido,
f_orders.cd_cliente,
f_items.cd_produto,
f_items.vlr_total,
f_items.dt_pedido,
d_products.ds_nivel,
d_products.ds_cor,
d_products.ds_tamanho,
d_products.ds_marca,
d_products.ds_categoria,
d_products.ds_colecao,
d_products.fx_preco,
d_products.ds_tipo
FROM 
public.f_items
left join public.d_products on f_items.cd_produto = d_products.cd_produto
left join public.f_orders on f_orders.cd_pedido = f_items.cd_pedido
inner join public.sts_clients_stats on sts_clients_stats.cd_cliente = f_orders.cd_cliente
where 
f_orders.dt_pedido >= '2021-01-01'
)
select
base_dados.cd_cliente,
count(distinct base_dados.cd_produto) as count_cd_produto,
count(distinct base_dados.ds_nivel) as count_ds_nivel,
count(distinct base_dados.ds_cor) as count_ds_cor,
count(distinct base_dados.ds_tamanho) as count_ds_tamanho,
count(distinct base_dados.ds_marca) as count_ds_marca,
count(distinct base_dados.ds_categoria) as count_ds_categoria,
count(distinct base_dados.ds_colecao) as count_ds_colecao,
count(distinct base_dados.fx_preco) as count_fx_preco,
count(distinct base_dados.ds_tipo) as count_ds_tipo
FROM 
base_dados
group by 
base_dados.cd_cliente
;
"""


cursor: redshift_connector.Cursor = conn.cursor()
cursor.execute(sql_count_produtos_cli)
sql_count_produtos_cli: pd.DataFrame = cursor.fetch_dataframe() 
dataset_count_produtos_cli = pd.DataFrame(sql_count_produtos_cli)
dataset_count_produtos_cli.to_csv('outros_dados/count_produtos_cli.csv', index=False,encoding='utf-8', header=True)
cursor.close()
conn.close()

## 1 - Carregando o Dataset

In [ ]:
# Carregando o dataste
dataset = pd.read_csv('outros_dados/dataset_analise_BTYD.csv')
#dataset.columns

In [ ]:
# Passando os dados para Dataframe e verificando se possui valores NA
df_analise = pd.DataFrame(dataset)
#df_analise.info()

## 2 - Tratamento dos dados

In [ ]:
# Nesta fase vamos fazer a filtragem dos dados apenas para clientes que possuem frequência de compra > 0
# Vamos também fazer o tratamento da coluna data
df_analise['dt_pedido'] = df_analise['dt_pedido'] = pd.to_datetime(df_analise['dt_pedido'], format='%Y-%m-%d')

In [ ]:
# Filtando o dataset apenas para valores positivos
df_analise.loc[:, 'vlr_real'] = df_analise.loc[:, 'vlr_real'].where(df_analise.loc[:, 'vlr_real'] > 0)
df_analise = df_analise.dropna()

In [ ]:
# Fazendo a filtragem necessária das colunas e atribuindo um novo nome ao dataset
df = df_analise.loc[:, ('cd_cliente','dt_pedido','vlr_real')]
#df

In [ ]:
# Vamos buscar os códigos de cliente únicos
#print(df['cd_cliente'].nunique())

In [ ]:
# Vamos verificar os pedidos feitos
ultimo_pedido = df['dt_pedido'].max()
#ultimo_pedido
#print(df[(df['cd_cliente']==68520)])

## 3 - Criando a matrix RFM (Recency, Frequency, Monetary Value)
Agora que vai começar o nosso trabalho de análise de probabilidade, com base no pacote lifetimes

In [ ]:
# Vamos usar a função do pacote lifetime para criar a matrix RFM automaticamente
# Com base nas 3 colunas que ja temos filtradas no nosso dataset
from lifetimes.utils import summary_data_from_transaction_data
df_btyd = summary_data_from_transaction_data(df, 'cd_cliente', 'dt_pedido', monetary_value_col='vlr_real')
#df_btyd

<br/> cd_cliente - Código do cliente <br/>

<br/> Frequency - representa o número de compras repetidas que o cliente fez. Isso significa que é um a menos que o número total de compras. Na verdade, isso está um pouco errado. É a contagem de períodos de tempo distintos em que o cliente fez uma compra, ou meses ou anos.
Portanto, se estiver usando dias como unidades, é a contagem de dias distintos em que o cliente fez uma compra. <br/>

<br/> T - representa a idade do cliente em qualquer unidade de tempo escolhida. Isso é igual à duração entre a primeira compra de um cliente e o final do período em estudo. <br/>

<br/> Recency - representa a idade do cliente no momento em que fez suas compras mais recentes. Isso é igual à duração entre a primeira compra de um cliente e sua última compra. (Portanto, se eles fizeram apenas 1 compra, o tempo para retorno é 0.). <br/>

In [ ]:
# Vamos criar um histograma para ver a distribuição de frequencia de compra dos nossos clientes
# Também vamos imprimir algumas métricas sobre a frequencia de compra de nossos clinetes
#%matplotlib inline
#df_btyd['frequency'].plot(kind = 'hist', bins = 50)


#print(df_btyd['frequency'].describe())
#just_one_buy = round(sum(df_btyd['frequency'] == 0)/float(len(df_btyd))*(100),2)
#print("Percentual de clientes que compraram apenas uma vez:", just_one_buy,"%")

## 4 - Fazendo fit do modelo e criando alguns gráficos para análise RFM

In [ ]:
# Treinando o modelo
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=1)
bgf.fit(df_btyd['frequency'], df_btyd['recency'], df_btyd['T'])

In [ ]:
# Avaliando o modelo
from lifetimes.utils import calibration_and_holdout_data

summary_cal_holdout = calibration_and_holdout_data(df_analise, 'cd_cliente', 'dt_pedido',
                                        calibration_period_end='2021-09-01',
                                        observation_period_end='2021-12-31' )
#print(summary_cal_holdout.head())

In [ ]:
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases

bgf.fit(df_btyd['frequency'], df_btyd['recency'], df_btyd['T'])
#plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout)

In [ ]:
# Vamos vizualizar um gráfico de Frequencia / Recencia matrix
# Vou optar por fazer a importação apenas no momento do usa para deixar claro o uso de cada import
#%matplotlib inline
#from lifetimes.plotting import plot_frequency_recency_matrix
#fig = plt.figure(figsize=(12,8))
#plt.style.use('fivethirtyeight')
#plot_frequency_recency_matrix(bgf, cmap = 'gnuplot', T = 30)


### Interpretando o gráfico plot_frequency_recency_matrix
Podemos ver que os clientes com maior probilidade de compra, são aqueles que mesmo com uma alta recencia possuem uma alta frequencia de compra, olhando a parte amarela do gráfico.



In [ ]:
# Criando um gráfico de probabilidade de vida do cliente
#from lifetimes.plotting import plot_probability_alive_matrix
#fig = plt.figure(figsize=(12,8))
#plt.style.use('fivethirtyeight')
#plot_probability_alive_matrix(bgf, cmap = 'gnuplot')

### Interpretando o gráfico plot_probability_alive_matrix
Os clientes bons são aqueles que podem possuir uma grande recencia e grande frequencia de compras, queles com pouca recencia mas uma boa frequencia ou clientes que compraram recentemente.Esse gráfico delimita um range temporal da primeira compra e ultima e dentro deste range ele verifica a frequencia de compras e com base nisto ele tem a probabilidade de vida do cliente. Por isso clientes com longo perioso e alta frequencia são clientes com baixa chance de cancelamento dos serviços(ou seja morte).

## 5 - Probabilidade transações futuras com modelo Gamma-Gamma

Com o modelo GammaGamma vamos ter a previsão do valor de vida do cliente, número esperado de transações e receita esperada. Um ponto importante para o modelo GammaGamma é que ele parte da premissa de que não a correlação entre valor monetário e recencia, o que pode não ser a realidade em alguns casos, pois o cliente pode comprar grandes quantidades para diminuir o número de transações e custos agregados ao mesmo

In [ ]:
# Vamos usar os dados histórios e prever as transações dos próximos 30/60/90 dias dos top 10 clientes
# Estamos utilizando o numero esperado condicional 

df_btyd['pred_num_trans_30'] = round(bgf.conditional_expected_number_of_purchases_up_to_time(30, 
                                                                                          df_btyd['frequency'],
                                                                                         df_btyd['recency'],
                                                                                         df_btyd['T']),2)


#df_btyd.sort_values(by = 'pred_num_trans_30', ascending=False).head(10).reset_index()

### Interpretando conditional_expected_number_of_purchases_up_to_time
A coluna pred_num_trans vai informar o numero de transações esperada nos próximos 30 dias dos nossos top 10 clientes com base nos calculos do modelo

In [ ]:
# Vamos avaliar nosso modelo verificar se nossas previsões fazem sentido
#from lifetimes.plotting import plot_period_transactions
#fig = plt.figure(figsize=(12,8))
#plot_period_transactions(bgf)

### Interpretando plot_period_transactions
Podemos ver que o nosso modelo não está ruim, pois as barras de previsão X real estão bem próximas

## 6 - Fazendo previsão de transações futuras de um único cliente com .predict

In [ ]:
# Previsão de transações futuras de um único cliente
#t = 30
#individuo = df_btyd.loc[33816]
#print('Número de transações esperadas nos próximos 30 dias:',bgf.predict(t, 
                                                                         #individuo['frequency'],
                                                                         #individuo['recency'],
                                                                         #individuo['T']),'transações')

In [ ]:
# Validando o modelo Gamma Gamma vamos verificar a correlação entre o valor monetário da transação com a frequencia de compras
# Podemos perceber que existe uma correlação entre ambos, porém irei manter o estudo do GammaGamma
# Mais para frente verificaremos a possibilidade de usar um modelo que considera a relação entre ambos
#df_btyd[['monetary_value','frequency']].corr()

## 7 - Filtrando e analisando clientes que fizeram mais de uma compra


In [ ]:
shortlisted_costumers = df_btyd[df_btyd['frequency']>0]
#print(shortlisted_costumers.head().reset_index())
#print('--------------------------------------------------')
#print('Número total de clientes com freq > 0:',len(shortlisted_costumers))

## 8 - Treinando o modelo Gamma Gamma considerando o valor monetário

In [ ]:
# Fazendo importação do modelo e treinando com os dados filtrados de clientes que repetiram compra pelo menos 1 vez
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef=0.0)
ggf.fit(shortlisted_costumers['frequency'],
       shortlisted_costumers['monetary_value'] ,q_constraint = True)
print(ggf)

## 9 - Prevendo a média de valor de transação para cada cliente

In [ ]:
# Vamos fazer a previsão por cliente e imprimir os 10 primeiros resultados
print(ggf.conditional_expected_average_profit(
df_btyd['frequency'],
df_btyd['monetary_value']))

In [ ]:
# Agora vamos incluir no dataset a coluna com o valor médio de transação previsto pelo modelo
# Essa é a media esperada por transação
df_btyd['pred_trans_avg_value'] = round(ggf.conditional_expected_average_profit(
    df_btyd['frequency'],
    df_btyd['monetary_value']),2)


Com base nestas previsões podemos criar estratégias para atingir os clientes com maior potencial de valor monetário e também otimizar aqueles clientes com potencial médio ou baixo.

## 10 - Calculando o CLV dos clientes nos próximos 12 meses

#### frequency :
Representa o número de compras repetidas que o cliente fez. Isso significa que é um a menos que o número total de compras. Na verdade, isso está um pouco errado. É a contagem de períodos de tempo em que o cliente fez uma compra. Portanto, se estiver usando dias como unidades, é a contagem de dias em que o cliente fez uma compra.
#### T : 
Representa a idade do cliente em quaisquer unidades de tempo escolhidas, Isso é igual à duração entre a primeira compra de um cliente e o final do período em estudo.
#### recency :
Representa a idade do cliente no momento em que fez suas compras mais recentes. Isso é igual à duração entre a primeira compra de um cliente e sua última compra. (Portanto, se eles fizeram apenas 1 compra, o tempo para retorno é 0.)
#### monetary_value :
Representa o valor médio das compras de um determinado cliente. Isso é igual à soma de todas as compras de um cliente dividida pelo número total de compras. Observe que o denominador aqui é diferente do frequencydescrito acima.

In [ ]:
# Vamos criar uma nova coluna CLV no nosso dataframe
from lifetimes.utils import _customer_lifetime_value

df_btyd['CLV'] = round(ggf.customer_lifetime_value(
    bgf, # Modelo para prever o numero de transações futuras
    df_btyd['frequency'],
    df_btyd['recency'],
    df_btyd['T'],
    df_btyd['monetary_value'],
    time = 12, # 12 meses
    discount_rate = 0.01 # Parametro para ajuste de variações de valor futuras
),2)
df_btyd = df_btyd.sort_values(by='CLV', ascending=False).reset_index()
df_btyd

In [ ]:
# Calculando a probabilidade do cliente estar vivo

df_btyd['prob_alive'] = bgf.conditional_probability_alive(df_btyd['frequency'],df_btyd['recency'],df_btyd['T'])

In [ ]:
df_btyd.shape

Com está informação podemos listar os clientes provavelmente mais ou menos rentaveis e direcionar nossos esforços para otimiza-los, aumentando assim nossa receita.

## 11 - Gerando um novo dataset com base nos calculos feitos

In [ ]:
# Gerando o dataset para análise da probabilidade de compra dos clientes

df_btyd.to_csv('BTYD_probs.csv', index=False, header = True)

In [ ]:
df_btyd.sort_values('prob_alive',ascending=False)